# import modules

In [ ]:
import os
import requests
import pandas as pd
import json

from datetime import datetime

In [14]:
token = 'EAACEdEose0cBAMy35Rs1a6PrnNS3ZC1kPH4HVQLOhtn7cn8jiBjUTaBQHdbwiJhST7U2PIn7PmbxLz9n8ZCaDlTPdDqER15SZBptDFcKmEuALhTsvZC9zQVGni6VrT2bYZCgMubxbLH38j4kny4wWN8FIPqVvx9yJHOXhFwoBwFDKJg0H87NI14nWKN1nW7iztSbYnDHwp1FLqFZC7ARA9'

## source from fb 粉專：DLive長沙周末聯盟

In [32]:
"""
%7B => {
%7D => }
%2C => ,
reference: https://www.w3schools.com/tags/ref_urlencode.asp
"""

url = 'https://graph.facebook.com/v2.9/DLIVECJbasket?fields=albums.limit({})%7Bphotos.limit({})%7Balbum,images%7D%7D&access_token={}'.format(
    100,
    100,
    token)

response = requests.get(url)
html = json.loads(response.text) 

In [33]:
#抓出有幾本相簿 to find the numbers of albums
albums_num = html['albums']['data']
len(albums_num)

50

In [41]:
# parsing the html file...
albums_num[23]['photos']['data'][6]['images'][5]['source']

'https://scontent.xx.fbcdn.net/v/t1.0-0/p320x320/30704883_2109081332452754_4867429545419997184_n.jpg?_nc_cat=0&oh=935ed66351bd7a43f2e3ebe528b7eadf&oe=5B68A60D'

In [35]:
#list the index, albums_name, photos_in_albums
for i in range(len(albums_num)):
    print(i, albums_num[i]['photos']['data'][0]['album']['name'], 'has :', len(albums_num[1]['photos']['data']), 'photos!')

0 Timeline Photos has : 55 photos!
1 0415中正週日第五季C組 Rhinos - 中租控股 has : 55 photos!
2 0415中正週日第五季C組 Rocknrolla - 一號下卡拉 has : 55 photos!
3 0415中正週日第五季C組 達芬奇 - CNT has : 55 photos!
4 0415中正週日第五季C組 金鈺銨 - Black Panther has : 55 photos!
5 0415中正週日第五季A組 VG cafe - 美商雷馬 has : 55 photos!
6 0415中正週日第五季A組 Homies - 土城億客隆 has : 55 photos!
7 0415中正週日第五季A組 帥宇國際 - DV-P力貓 has : 55 photos!
8 0415中正週日第五季A組 潔鍶沙城豹 - 大鼎旅遊 has : 55 photos!
9 0415中正週日第五季D組 EXTREME - 經典帥 has : 55 photos!
10 0415中正週日第五季B組 Ranger - 欣辰照明 has : 55 photos!
11 0415中正週日第五季D組 恆永投資 - 雅偲珍顏 has : 55 photos!
12 0415中正週日第五季D組 HOPE - SL111 has : 55 photos!
13 0414中正週六女子B組 Double P - Umm has : 55 photos!
14 0414中正週六女子B組 EMBA LADIES - 安聯i探索 has : 55 photos!
15 0414中正週六女子A組 DAME - 台大啊嚕嘪 has : 55 photos!
16 0414中正週六女子A組 NBS - 政妹隊 has : 55 photos!
17 0415 第二季和平館週日C組 Uncle - Pecado has : 55 photos!
18 0415第二季和平館週日B組 CHE System - Never1551 has : 55 photos!
19 0415第二季和平館週日B組 Lunzer - 4th Lane has : 55 photos!
20 0415第二季和平館週日D組 AK57ers - WED-D has : 5

In [36]:
#把所有爬到的imgs存進imgs_list裡 append all photos in imgs_list
#第一個index表示第幾本相簿，第二個index是第幾張，第三個是同張第幾個尺寸
imgs_list = []
for i in range(1, len(albums_num)):
    for j in range(len(albums_num[i]['photos']['data'])):
        imgs_list.append(albums_num[i]['photos']['data'][j]['images'][5]['source'])

In [37]:
len(imgs_list)

2624

In [38]:
imgs_list[2623]

'https://scontent.xx.fbcdn.net/v/t1.0-0/p320x320/29683757_2091475990879955_6311418045311275654_n.jpg?_nc_cat=0&oh=2ea2f55798c34c49a1bb5144efa21ac9&oe=5B68ADAE'

In [42]:
from PIL import Image
from bs4 import BeautifulSoup

In [46]:
#testing...
for i, n in enumerate(imgs_list[1882:1885]):
    print(i+1882, n)

1882 https://scontent.xx.fbcdn.net/v/t1.0-0/p320x320/29595077_2091601887534032_6427098634081748653_n.jpg?_nc_cat=0&oh=41348df8c5c02f7ae113a0c349cdba18&oe=5B685FEF
1883 https://scontent.xx.fbcdn.net/v/t1.0-0/p320x320/29694682_2091603944200493_5751625844858458319_n.jpg?_nc_cat=0&oh=98a6a9493726f62e35081ddb983969a8&oe=5B700A0C
1884 https://scontent.xx.fbcdn.net/v/t1.0-0/p320x320/29790246_2091603990867155_8398447190680111857_n.jpg?_nc_cat=0&oh=aeb555768e3e05f76f1102f48e090126&oe=5B539ED1


In [47]:
#save the photos in folder 'results'
results = os.path.abspath('results')
if not os.path.exists(results):
    os.makedirs(results)

for i, n in enumerate(imgs_list):
    img_resp = requests.get(n, stream=True)
    image = Image.open(img_resp.raw)
    filename = os.path.basename(str(i))
    #print('catch the filename {} and the real format is {}'.format(filename, image.format))
    
    real_filename = '{}.{}'.format(
        filename,
        image.format.lower()
    )
    save_filename = os.path.join(results, real_filename)
    #print('catch the real filename {}'.format(real_filename))
    
    image.save(save_filename)
    #print('save image at {}'.format(save_filename))

# 將train set 的檔名修改成s_num, d_num, l_num形式

In [3]:
shoot_path = '../dataset/train/shoot/'
dribble_path = '../dataset/train/dribble/'
layup_path = '../dataset/train/layup/'

shoot_file_names = [os.path.join(shoot_path, f) for f in os.listdir(shoot_path) if (f.endswith('g')) | (f.endswith('G'))]
dribble_file_names = [os.path.join(dribble_path, f) for f in os.listdir(dribble_path) if (f.endswith('g')) | (f.endswith('G'))]
layup_file_names = [os.path.join(layup_path, f) for f in os.listdir(layup_path) if (f.endswith('g')) | (f.endswith('G'))]

print('shoot has :', len(shoot_file_names))
print('dribble has :', len(dribble_file_names))
print('layup has :', len(layup_file_names))

shoot has : 945
dribble has : 967
layup has : 869


In [4]:
count = 1

for f in shoot_file_names:
    new_f = str(count) + '.jpeg'
    os.rename(f, os.path.join(shoot_path, 's' + new_f))
    count += 1

In [5]:
count = 1

for f in dribble_file_names:
    new_f = str(count) + '.jpeg'
    os.rename(f, os.path.join(dribble_path, 'd' + new_f))
    count += 1

In [6]:
count = 1

for f in layup_file_names:
    new_f = str(count) + '.jpeg'
    os.rename(f, os.path.join(layup_path, 'l' + new_f))
    count += 1